In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import *


Orig_test_data=pd.read_excel('/content/PROPRIETARY_INFORMATION_Orig_Holdout_Data.xlsx',sheet_name='Sheet1',)


def PMT(rate, nper,pv, fv=0, type=0):
    if rate!=0:
               pmt = (rate*(fv+pv*(1+ rate)**nper))/((1+rate*type)*(1-(1+ rate)**nper))
    else:
               pmt = (-1*(fv+pv)/nper)  
    return(pmt)


def IPMT(rate, per, nper,pv, fv=0, type=0):
  ipmt = -( ((1+rate)**(per-1)) * (pv*rate + PMT(rate, nper,pv, fv=0, type=0)) - PMT(rate, nper,pv, fv=0, type=0))
  return(ipmt)


def PPMT(rate, per, nper,pv, fv=0, type=0):
  ppmt = PMT(rate, nper,pv, fv=0, type=0) - IPMT(rate, per, nper, pv, fv=0, type=0)
  return(ppmt)

def amortisation_schedule(amount, annualinterestrate, paymentsperyear, years,report_date):

    df = pd.DataFrame({'ppmt' :[PPMT(annualinterestrate/paymentsperyear, i+1, paymentsperyear*years, amount) for i in range(paymentsperyear*years)],
                                 'ipmt' :[IPMT(annualinterestrate/paymentsperyear, i+1, paymentsperyear*years, amount) for i in range(paymentsperyear*years)]})
    df.ppmt=-1*df.ppmt
    df.ipmt=-1*df.ipmt
    df['eom'] = amount - np.cumsum(df.ppmt)
    df['eom'] =df['eom'].round(5)
    df['bom'] = df['eom'] + df.ppmt
    df['mob'] = [ i for i in range(1,years*12+1)]
    use_date = datetime.strptime(report_date, '%b%Y')
    df['report_date'] = [(use_date+relativedelta(months=+i)).strftime('%Y%m') for i in range(1,years*12+1)]
    return(df)
#amortisation_schedule(7500, 0.1075, 12, 3)

dataF=pd.DataFrame()
for index, row in Orig_test_data.head(3).iterrows():
    
    f=amortisation_schedule(row['loan_amount'], row['int_rate']/100, 12, int(row['term']/12),'APR2016')
    f['unique_id']=row['unique_id']
    #report
    dataF=pd.concat([dataF,f])
    
#dataF is the test data to be used for enerating probabilities

,ppmt,ipmt,eom,bom,mob,report_date,unique_id
0,257.117731,522.433333,27742.88227,28000.000001,1,201605,A-223264-1
1,261.915119,517.635945,27480.96715,27742.882269,2,201606,A-223264-1
2,266.802019,512.749045,27214.16513,27480.967149,3,201607,A-223264-1
3,271.780100,507.770964,26942.38503,27214.165130,4,201608,A-223264-1
4,276.851064,502.700001,26665.53397,26942.385034,5,201609,A-223264-1
...,...,...,...,...,...,...,...
31,206.192170,4.611299,833.95048,1040.142650,32,201812,A-640827-3
32,207.106289,3.697180,626.84419,833.950479,33,201901,A-640827-3
33,208.024460,2.779009,418.81973,626.844190,34,201902,A-640827-3
34,208.946702,1.856767,209.87303,418.819732,35,201903,A-640827-3
